In [ ]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

BASE = Path('/content/drive/MyDrive/censorly-dataset/Phobic')


Mounted at /content/drive


In [ ]:
!pip -q install --upgrade "ultralytics>=8.3.0" pillow opencv-python tqdm


In [ ]:
# ============================================================
# PHOBIC — GOOD RECIPE (YOLO11) + DRIVE SAVE + STAGE-2 FIX
#  - v8 ağırlığı yerine yolo11m.pt kullanır (torch.utils.serialization hatasını çözer)
#  - Stage-1 & Stage-2 doğrudan Drive'a kaydeder
#  - Stage-2'de duplicate arg ÇAKIŞMASI YOK
# ============================================================

# --- 0) Kurulum / Sürüm Fix ---
import sys, subprocess, time, yaml
from pathlib import Path

def pip_install(args):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + args)

# PyTorch 2.6+ unpickling değişikliğinden kaçın: 2.5.1'e sabitle (gerekirse)
try:
    import torch
    v = tuple(int(x) for x in torch.__version__.split("+")[0].split(".")[:2])
    if v >= (2,6):
        raise ImportError("Force downgrade")
except Exception:
    pip_install(["--extra-index-url","https://download.pytorch.org/whl/cu121","torch==2.5.1","torchvision==0.20.1","torchaudio==2.5.1"])
    import IPython; IPython.get_ipython().kernel.do_shutdown(True)

# Ultralytics
pip_install(["ultralytics==8.3.50"])

# --- 1) Drive & Yol Tanımları ---
from google.colab import drive
try: drive.mount("/content/drive")
except: pass

ROOT = Path("/content/drive/MyDrive/censorly-dataset")
RUNS_DIR = Path("/content/drive/MyDrive/censorly-runs"); RUNS_DIR.mkdir(parents=True, exist_ok=True)
ts = time.strftime("%Y%m%d_%H%M%S")

SETS = {
    "CLOWN_TRUE":  ROOT/"Phobic/Clown/Clown-True/images",
    "CLOWN_FALSE": ROOT/"Phobic/Clown/Clown-False/images",  # negatif (insan) img'ler, boş .txt
    "SPIDER_TRUE": ROOT/"Phobic/Spider/Spider-True/images",
    "SNAKE_TRUE":  ROOT/"Phobic/Snake/Snake-True/images",
}

def split_path_or_fallback(base_images_dir: Path, split: str) -> str:
    p = base_images_dir / split
    if p.exists(): return str(p)
    if split == "val":
        t = base_images_dir / "test"
        if t.exists():
            print(f"[INFO] {base_images_dir}/val bulunamadı -> val=test yönlendirmesi uygulanıyor.")
            return str(t)
    print(f"[UYARI] {p} bulunamadı."); return str(p)

train_dirs = [
    str(SETS["CLOWN_TRUE"]/"train"),
    str(SETS["CLOWN_FALSE"]/"train"),
    str(SETS["SPIDER_TRUE"]/"train"),
    str(SETS["SNAKE_TRUE"]/"train"),
]
val_dirs = [
    split_path_or_fallback(SETS["CLOWN_TRUE"],  "val"),
    split_path_or_fallback(SETS["CLOWN_FALSE"], "val"),
    split_path_or_fallback(SETS["SPIDER_TRUE"], "val"),
    split_path_or_fallback(SETS["SNAKE_TRUE"],  "val"),
]
test_dirs = [
    str(SETS["CLOWN_TRUE"]/"test"),
    str(SETS["CLOWN_FALSE"]/"test"),
    str(SETS["SPIDER_TRUE"]/"test"),
    str(SETS["SNAKE_TRUE"]/"test"),
]

data_yaml = {
    "path": ".",
    "train": train_dirs,
    "val":   val_dirs,
    "test":  test_dirs,
    "names": ["clown","spider","snake"],
    "nc": 3
}
Path("data_phobic.yaml").write_text(yaml.safe_dump(data_yaml, sort_keys=False), encoding="utf-8")
print("=== data_phobic.yaml ===\n", Path("data_phobic.yaml").read_text())

# --- 2) Hiperparametreler (ÖNCEKİ İYİ REÇETE) ---
hyp = {
  "degrees": 10.0, "translate": 0.08, "scale": 0.6, "shear": 0.0, "perspective": 0.000,
  "fliplr": 0.5, "flipud": 0.0,
  "hsv_h": 0.015, "hsv_s": 0.6, "hsv_v": 0.4,
  "mosaic": 1.0, "mixup": 0.1, "copy_paste": 0.0, "erasing": 0.0, "crop_fraction": 1.0,
  "box": 7.5, "cls": 0.5, "dfl": 1.5, "label_smoothing": 0.0,
  "lr0": 0.01, "lrf": 0.01, "momentum": 0.937, "weight_decay": 0.0005,
  "warmup_epochs": 3.0, "warmup_momentum": 0.8, "warmup_bias_lr": 0.1
}
Path("hyp_phobic.yaml").write_text(yaml.safe_dump(hyp, sort_keys=False), encoding="utf-8")

# Stage-2 için (mosaic kapalı + LR düşür)
hyp_fine = hyp.copy()
hyp_fine.update({"mosaic": 0.0, "mixup": 0.0, "hsv_s": 0.4, "hsv_v": 0.3, "degrees": 5.0, "scale": 0.4, "lr0": 0.003})
Path("hyp_phobic_fine.yaml").write_text(yaml.safe_dump(hyp_fine, sort_keys=False), encoding="utf-8")

print("=== hyp_phobic.yaml ===\n", Path("hyp_phobic.yaml").read_text())
print("=== hyp_phobic_fine.yaml ===\n", Path("hyp_phobic_fine.yaml").read_text())

# --- 3) Eğitim (Drive'a kaydet) ---
from ultralytics import YOLO

MODEL  = "yolo11m.pt"   # <--- v8 yerine YOLO11 kullan (torch.utils.serialization hatası çözülür)
IMGSZ  = 1024
BATCH  = 16
DEVICE = 0

# Hiperleri yükle
with open("hyp_phobic.yaml", "r") as f:      hyp_loaded = yaml.safe_load(f) or {}
with open("hyp_phobic_fine.yaml", "r") as f: hyp_fine_loaded = yaml.safe_load(f) or {}

# Stage-1 → Drive
overrides_stage1 = dict(
    data="data_phobic.yaml",
    imgsz=IMGSZ,
    epochs=80,
    batch=BATCH,
    device=DEVICE,
    cos_lr=True,
    workers=8,
    cache="ram",
    close_mosaic=10,
    patience=20,
    save_period=-1,
    project=str(RUNS_DIR),
    name=f"phobic_stage1_{ts}",
    **hyp_loaded,
)

print("\n[TRAIN-1] Strong Aug başlıyor → Drive")
model = YOLO(MODEL)
res1 = model.train(**overrides_stage1)
print("[TRAIN-1] Tamamlandı.")

# Stage-1 best.pt (Drive içinden)
stage1_dir = RUNS_DIR / f"phobic_stage1_{ts}"
best_pt = stage1_dir / "weights" / "best.pt"
assert best_pt.exists(), f"best.pt bulunamadı: {best_pt}"
print(f"[INFO] best.pt: {best_pt}")

# Stage-2 → Drive (mosaic kapalı) — duplicate arg YOK (lr0 ve mosaic hyp_fine içinde)
overrides_stage2 = dict(
    data="data_phobic.yaml",
    imgsz=IMGSZ,
    epochs=25,
    batch=BATCH,
    device=DEVICE,
    cos_lr=True,
    workers=8,
    cache="ram",
    project=str(RUNS_DIR),
    name=f"phobic_stage2_{ts}",
    save_period=-1,
    **hyp_fine_loaded,
)

print("\n[TRAIN-2] Fine (mosaic=0) başlıyor → Drive")
model = YOLO(str(best_pt))
res2 = model.train(**overrides_stage2)
print("[TRAIN-2] Tamamlandı.")

# Özet
stage2_dir = RUNS_DIR / f"phobic_stage2_{ts}"
print("\n=== EĞİTİM BİTTİ ===")
print("Stage-1:", stage1_dir)
print("  ↳ best.pt:", stage1_dir/"weights/best.pt")
print("Stage-2:", stage2_dir)
print("  ↳ best.pt:", stage2_dir/"weights/best.pt")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] /content/drive/MyDrive/censorly-dataset/Phobic/Clown/Clown-True/images/val bulunamadı -> val=test yönlendirmesi uygulanıyor.
[INFO] /content/drive/MyDrive/censorly-dataset/Phobic/Clown/Clown-False/images/val bulunamadı -> val=test yönlendirmesi uygulanıyor.
[INFO] /content/drive/MyDrive/censorly-dataset/Phobic/Spider/Spider-True/images/val bulunamadı -> val=test yönlendirmesi uygulanıyor.
[INFO] /content/drive/MyDrive/censorly-dataset/Phobic/Snake/Snake-True/images/val bulunamadı -> val=test yönlendirmesi uygulanıyor.
=== data_phobic.yaml ===
 path: .
train:
- /content/drive/MyDrive/censorly-dataset/Phobic/Clown/Clown-True/images/train
- /content/drive/MyDrive/censorly-dataset/Phobic/Clown/Clown-False/images/train
- /content/drive/MyDrive/censorly-dataset/Phobic/Spider/Spider-True/images/train
- /content/drive/MyDrive/censorly-dataset/Phobic/Snake/Snak

100%|██████████| 38.8M/38.8M [00:00<00:00, 215MB/s]


New https://pypi.org/project/ultralytics/8.3.205 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=data_phobic.yaml, epochs=80, time=None, patience=20, batch=16, imgsz=1024, save=True, save_period=-1, cache=ram, device=0, workers=8, project=/content/drive/MyDrive/censorly-runs, name=phobic_stage1_20251006_090718, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_b

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

...

      25/25      21.9G     0.9358      0.802      1.554          3       1024: 100%|██████████| 154/154 [00:44<00:00,  3.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.98it/s]

                   all        301        252      0.806       0.68      0.756      0.485



25 epochs completed in 0.335 hours.
Optimizer stripped from /content/drive/MyDrive/censorly-runs/phobic_stage2_20251006_090718/weights/last.pt, 40.5MB
Optimizer stripped from /content/drive/MyDrive/censorly-runs/phobic_stage2_20251006_090718/weights/best.pt, 40.5MB

Validating /content/drive/MyDrive/censorly-runs/phobic_stage2_20251006_090718/weights/best.pt...
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11m summary (fused): 303 layers, 20,032,345 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  3.89it/s]


                   all        301        252      0.817      0.693      0.764      0.486
                 clown         74         86      0.765      0.454      0.594      0.339
                spider         80         81      0.741      0.671      0.757       0.39
                 snake         80         85      0.945      0.953      0.941       0.73
Speed: 0.3ms preprocess, 3.8ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to /content/drive/MyDrive/censorly-runs/phobic_stage2_20251006_090718
[TRAIN-2] Tamamlandı.

=== EĞİTİM BİTTİ ===
Stage-1: /content/drive/MyDrive/censorly-runs/phobic_stage1_20251006_090718
  ↳ best.pt: /content/drive/MyDrive/censorly-runs/phobic_stage1_20251006_090718/weights/best.pt
Stage-2: /content/drive/MyDrive/censorly-runs/phobic_stage2_20251006_090718
  ↳ best.pt: /content/drive/MyDrive/censorly-runs/phobic_stage2_20251006_090718/weights/best.pt
